# ***XỬ LÝ ẢNH***
    Xoá các file có không phải là ảnh (ví dụ: .avi)
    Chuyển đổi các file ảnh như: .jpg, .jpe sang các file ảnh .png
    Resize các ảnh về cùng kích thước

In [1]:
import pandas as pd
from PIL import Image
import os

In [2]:
df = pd.read_excel("data_cleaned_2.xlsx")

> Convert ảnh

In [3]:
extensions = df['image'].dropna().apply(lambda x: os.path.splitext(x)[1].lower())
extension_counts = extensions.value_counts()

print("📂 Available image extensions:")
print(extension_counts)

📂 Available image extensions:
image
.jpg    380
.png     93
.avi     14
.jpe      9
Name: count, dtype: int64


In [4]:
df = df[~df['image'].str.lower().str.endswith('.avi', na=False)]

In [5]:
INPUT_FOLDER = "images"
OUTPUT_FOLDER = "images_png"

count = 0
error_count = 0
converted_paths = {}

In [6]:
for idx, row in df.iterrows():
    image_path = row['image']
    product_id = str(row['product_id'])

    if pd.isna(image_path) or pd.isna(product_id):
        continue

    # Tách tên file và thư mục con
    image_path = image_path.strip().replace("\\", "/")
    rel_dir = os.path.dirname(image_path)
    filename = os.path.basename(image_path)

    src_path = os.path.join(INPUT_FOLDER, rel_dir, filename)
    dst_dir = os.path.join(OUTPUT_FOLDER, rel_dir)
    dst_filename = f"{product_id}.png"
    dst_path = os.path.join(dst_dir, dst_filename)

    try:
        os.makedirs(dst_dir, exist_ok=True)
        image = Image.open(src_path).convert("RGB")
        image.save(dst_path)
        converted_paths[product_id] = os.path.join(rel_dir, dst_filename).replace("\\", "/")
        print(f"Converted {src_path} -> {dst_path}")
        count += 1
    except Exception as e:
        print(f"Error {src_path}: {e}")
        error_count += 1

Converted images/Chuột/151794732_4849244250405221918.jpg -> images_png/Chuột/151794732.png
Converted images/Chuột/554918_6653251552233206651.jpg -> images_png/Chuột/554918.png
Converted images/Chuột/1662259_7147061146754508183.jpg -> images_png/Chuột/1662259.png
Converted images/Chuột/6431175_-2761762888004500487.png -> images_png/Chuột/6431175.png
Converted images/Chuột/17587170_7262822802616018413.PNG -> images_png/Chuột/17587170.png
Converted images/Chuột/68132909_-4974018489174899550.jpg -> images_png/Chuột/68132909.png
Converted images/Chuột/74553497_2868333515564354288.jpg -> images_png/Chuột/74553497.png
Converted images/Chuột/75770350_159568558306327625.JPG -> images_png/Chuột/75770350.png
Converted images/Chuột/76310047_5910686356014007222.png -> images_png/Chuột/76310047.png
Converted images/Chuột/104069839_-7493300662824614360.jpg -> images_png/Chuột/104069839.png
Converted images/Chuột/120441377_-8067432762384670889.png -> images_png/Chuột/120441377.png
Converted images/Chu

In [7]:
print(f"\nConversion complete: {count} images succeeded, {error_count} failed.")


Conversion complete: 482 images succeeded, 0 failed.


In [8]:
df['image'] = df['product_id'].astype(str).map(converted_paths)
df = df[df['image'].notna()].copy()

In [9]:
df.to_excel("data_converted.xlsx", index=False)
print("Saved 'data_converted.xlsx'")

Saved 'data_converted.xlsx'
